**Let's Learn SQLALCHEMY**


**EP1 start**

When you want to use SQLALCHEMY to connect to DB there are 3 layers:
<br>
1.DBAPI
<br>
2.Core
<br>
3.ORM
<br>

---

What is **DBAPI**?
<br>
DBAPI is like a driver , a standard that tell python libraries that if you want to connect to database you should adhere some rules.
<br>
also we can define it with python oop like this:

<pre>
from abc import ABC,abstractmethod
class DBAPI(ABC):
    @abstractmethod
    def connect(self):
        pass
    
    @abstractmethod
    def cusror(self):
        pass

class PSYCOPG(DBAPI):
    def connect(self):
        # implement psql logics
        return psql.connection
    def cusror(self):
        # implement psql logics
        return psql.cursor

class PYMYSQL(DBAPI):
    def connect(self):
        # implement mysql logics
        return mysql.connection
    def cusror(self):
        # implement mysql logics
        return mysql.cursor
</pre>
<br>
<br>
so 
<br>
DBAPI -> abstract class (interface)
<br>
.connect(),.cursor(),... -> abstractmethods
<br>
psycopg,pymysql,sqlite -> concrete class
